## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-08-05-35-42 +0000,nyt,Why France’s Financial Woes Are Pushing Its Go...,https://www.nytimes.com/2025/09/07/business/fr...
1,2025-09-08-05-33-58 +0000,wapo,Australian woman gets life in prison for ‘mush...,https://www.washingtonpost.com/world/2025/09/0...
2,2025-09-08-05-17-27 +0000,nypost,Elderly California jewelry store owner assault...,https://nypost.com/2025/09/08/us-news/californ...
3,2025-09-08-05-12-14 +0000,bbc,'Suitcase murder' trial begins in New Zealand,https://www.bbc.com/news/articles/cn83jjx9pjvo...
4,2025-09-08-05-12-07 +0000,latimes,"David Baltimore, Nobel Prize-winning scientist...",https://www.latimes.com/obituaries/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
102,trump,20
149,ukraine,10
4,government,10
133,raid,10
34,new,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
172,2025-09-07-12-19-07 +0000,nypost,South Korean workers arrested in ICE raid at G...,https://nypost.com/2025/09/07/us-news/south-ko...,82
154,2025-09-07-15-08-17 +0000,nypost,Russian attack on Ukraine’s government HQ in l...,https://nypost.com/2025/09/07/world-news/russi...,74
120,2025-09-07-17-38-00 +0000,wsj,Russia’s Largest Drone Attack Yet Hits Ukraine...,https://www.wsj.com/world/russias-largest-dron...,59
112,2025-09-07-18-45-35 +0000,nyt,Russia Strikes Ukraine Government Building in ...,https://www.nytimes.com/2025/09/07/world/europ...,59
192,2025-09-07-09-34-43 +0000,cbc,Russia's largest air attack on Ukraine sets fi...,https://www.cbc.ca/news/world/russia-mass-assa...,58


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
172,82,2025-09-07-12-19-07 +0000,nypost,South Korean workers arrested in ICE raid at G...,https://nypost.com/2025/09/07/us-news/south-ko...
112,59,2025-09-07-18-45-35 +0000,nyt,Russia Strikes Ukraine Government Building in ...,https://www.nytimes.com/2025/09/07/world/europ...
171,38,2025-09-07-12-25-45 +0000,nyt,Prime Minister Shigeru Ishiba of Japan Says He...,https://www.nytimes.com/2025/09/07/world/asia/...
72,33,2025-09-07-22-27-19 +0000,nypost,Fan wrongly ID’d as ‘Phillies Karen’ who snatc...,https://nypost.com/2025/09/07/sports/fan-wrong...
2,32,2025-09-08-05-17-27 +0000,nypost,Elderly California jewelry store owner assault...,https://nypost.com/2025/09/08/us-news/californ...
203,32,2025-09-07-08-42-01 +0000,nypost,Pope Leo XIV names first two saints of his pon...,https://nypost.com/2025/09/07/world-news/pope-...
41,31,2025-09-08-00-22-05 +0000,nyt,"Australian Woman, Erin Patterson, Gets Life in...",https://www.nytimes.com/2025/09/07/world/austr...
154,29,2025-09-07-15-08-17 +0000,nypost,Russian attack on Ukraine’s government HQ in l...,https://nypost.com/2025/09/07/world-news/russi...
65,27,2025-09-07-22-38-27 +0000,nypost,Meet the NYC shop cat working for over a decad...,https://nypost.com/2025/09/07/us-news/meet-the...
54,26,2025-09-07-23-33-59 +0000,latimes,"City buys Griffith Park carousel, where Disney...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
